# Simple version of backward
z = x * y + y

In [19]:
class SimpleTensor:
    def __init__(self, data):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
    
    def __add__(self, other):
        out = SimpleTensor(self.data + other.data)
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        out = SimpleTensor(self.data * other.data)
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        self.grad = 1.0
        self._backward()

In [13]:
x = SimpleTensor(2)
y = SimpleTensor(3)
z = x * y
z.backward()

In [14]:
print(f'gradient dz/dz = {z.grad}')
print(f'gradient dz/dx = {x.grad}')
print(f'gradient dz/dy = {y.grad}')

gradient dz/dz = 1.0
gradient dz/dx = 3.0
gradient dz/dy = 2.0


# Basic Version

In [20]:
import numpy as np
class Tensor:
    def __init__(self, data, children = (), op = ''):
        self.data = np.array(data, dtype = float)
        self.grad = np.zeros_like(self.data)
        self._backward = lambda: None
        self._prev = set(children)
        self.op = op

    def __add__(self, other):
        out = Tensor(self.data + other.data, children = (self, other), op = '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        out = Tensor(self.data * other.data, children = (self, other), op = '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        self.grad = np.ones_like(self.data)
        topo = []
        visited = set()
        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)
        build(self)
        for v in reversed(topo):
            v._backward()


In [ ]:
x = Tensor(4)
y = Tensor(7)
z = x*y + y
z.backward()

In [31]:
print(f'gradient dz/dz = {z.grad}')
print(f'gradient dz/dx = {x.grad}')
print(f'gradient dz/dy = {y.grad}')

gradient dz/dz = 1.0
gradient dz/dx = 7.0
gradient dz/dy = 5.0
